In [1]:
import os

os.environ["CHECKPOINTS_PATH"] = "../checkpoints"

import subprocess
import tempfile
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import torch
from sqlalchemy.orm import Session
from src.api.db import engine
from src.api.models.gaze import GazePoint
from src.api.models.pydantic import SimRoomClassDTO
from src.api.repositories import simrooms_repo
from src.api.services import gaze_service, simrooms_service, labeling_service
from src.config import TOBII_GLASSES_FPS, VIEWED_RADIUS, TOBII_GLASSES_RESOLUTION
from src.api.utils import image_utils
from src.utils import (
    extract_frames_to_dir,
)
from tqdm import tqdm

from experiment.settings import (
    FULLY_LABELED_RECORDINGS,
    GROUND_TRUTH_PATH,
    LABELING_VALIDATION_VIDEOS_PATH,
    SIMROOM_ID,
)

# Create the ground truth dataset

In [2]:
print(np.load("data/labeling_results/2/1/0.npz").files)

['mask', 'box', 'roi', 'class_id', 'frame_idx']


In [3]:
def get_viewed_annotations_per_frame(
    cal_rec_id: int,
    tracked_classes: list[SimRoomClassDTO],
    gaze_position_per_frame: dict[int, GazePoint],
    video_resolution: tuple[int, int] = TOBII_GLASSES_RESOLUTION,
):
    # Gather all annotation paths for each annotated frame
    annotations_per_frame: dict[int, list[Path]] = {}
    for anno_class in tracked_classes:
        annotation_paths = labeling_service.get_class_tracking_results(
            calibration_id=cal_rec_id, class_id=anno_class.id
        )

        print(
            f"Processing {len(annotation_paths)} annotations for class {anno_class.class_name}"
        )
        for annotation_path in tqdm(annotation_paths):
            frame_idx = int(annotation_path.stem)

            annotation_file = np.load(annotation_path)
            mask = annotation_file["mask"]
            x1, y1, x2, y2 = annotation_file["box"]

            # Put the mask in a tensor of the same size as the video frame
            mask_full = np.zeros(video_resolution, dtype=np.uint8)
            mask_full[y1:y2, x1:x2] = mask
            mask_full_torch = torch.from_numpy(mask_full)

            gaze_position = gaze_position_per_frame.get(frame_idx)
            if gaze_position is None:
                continue

            if gaze_service.mask_was_viewed(mask_full_torch, gaze_position):
                if frame_idx not in annotations_per_frame:
                    annotations_per_frame[frame_idx] = []

                annotations_per_frame[frame_idx].append(annotation_path)

    return annotations_per_frame

In [4]:
def draw_validation_video_frames(
    frames: list[Path],
    annotations_per_frame: dict[int, list[Path]],
    gaze_position_per_frame: dict[int, GazePoint],
    tracked_classes: list[SimRoomClassDTO],
):
    class_id_to_annotated_class = {
        anno_class.id: anno_class for anno_class in tracked_classes
    }

    # Iterate over frames and draw the annotations on them if they exist
    for frame in tqdm(frames, desc="Drawing annotations on frames"):
        frame_idx = int(frame.stem)
        frame_img = cv2.imread(str(frame))

        if annotations_per_frame.get(frame_idx) is not None:
            for annotation_path in annotations_per_frame[frame_idx]:
                annotation_file = np.load(annotation_path)
                class_id = int(annotation_file["class_id"])
                x1, y1, x2, y2 = annotation_file["box"]
                mask = annotation_file["mask"]

                # Squeeze mask if it has an extra dimension
                if mask.ndim == 3 and mask.shape[0] == 1:
                    mask = mask[0]
                if mask.dtype != bool:
                    mask = mask.astype(bool)

                color = class_id_to_annotated_class[class_id].color
                class_name = class_id_to_annotated_class[class_id].class_name
                box = (x1, y1, x2, y2)

                frame_img = image_utils.draw_mask(frame_img, mask, box)
                frame_img = image_utils.draw_labeled_box(
                    frame_img, box, class_name, color
                )

        # Draw the gaze point on the frame
        gaze_position = gaze_position_per_frame.get(frame_idx)
        if gaze_position is not None:
            gaze_x, gaze_y = gaze_position
            cv2.circle(
                frame_img,
                (int(gaze_x), int(gaze_y)),
                radius=VIEWED_RADIUS,
                color=(0, 0, 255),
                thickness=2,
            )

        # Save the modified image back to its original location
        cv2.imwrite(str(frame), frame_img)

In [5]:
if not LABELING_VALIDATION_VIDEOS_PATH.exists():
    os.makedirs(LABELING_VALIDATION_VIDEOS_PATH)
else:
    for file in LABELING_VALIDATION_VIDEOS_PATH.glob("*.mp4"):
        # os.remove(file)
        pass

if GROUND_TRUTH_PATH.exists():
    GROUND_TRUTH_PATH.unlink()

CREATE_VALIDATION_VIDEO = True
gt_rows = []
for recording_id in FULLY_LABELED_RECORDINGS:
    with Session(engine) as session:
        calibration_recording = simrooms_repo.get_calibration_recording(
            db=session,
            simroom_id=SIMROOM_ID,
            recording_id=recording_id,
        )
        tracked_classes = simrooms_service.get_tracked_classes(
            db=session, calibration_id=calibration_recording.id
        )

        trial_recording_path = calibration_recording.recording.video_path
        gaze_data_path = calibration_recording.recording.gaze_data_path

    # Extract video frames
    print(f"Extracting video frames for {recording_id}")
    frames, _ = simrooms_service.extract_tmp_frames(
        recording_id=recording_id,
    )

    # Load and preprocess gaze points
    print(f"Loading gaze data for {recording_id}")
    gaze_position_per_frame = gaze_service.get_gaze_position_per_frame(
        recording_id=recording_id,
        frame_count=len(frames),
    )

    # Get all annotations that were viewed
    print(f"Getting viewed annotations for {recording_id}")
    annotations_per_frame = get_viewed_annotations_per_frame(
        cal_rec_id=calibration_recording.id,
        tracked_classes=tracked_classes,
        gaze_position_per_frame=gaze_position_per_frame,
    )

    # Build the ground truth DataFrame
    print(f"Building ground truth for {recording_id}")
    for frame_idx, annotation_paths in annotations_per_frame.items():
        for annotation_path in annotation_paths:
            annotation_file = np.load(annotation_path)
            class_id = int(annotation_file["class_id"])
            mask_area = np.sum(annotation_file["mask"])
            x1, y1, x2, y2 = annotation_file["box"]
            roi = annotation_file["roi"]

            laplacian_variance = cv2.Laplacian(roi, cv2.CV_64F).var()

            gt_rows.append({
                "recording_id": recording_id,
                "frame_idx": frame_idx,
                "class_id": class_id,
                "mask_area": mask_area,
                "laplacian_variance": laplacian_variance,
                "x1": x1,
                "y1": y1,
                "x2": x2,
                "y2": y2,
            })

    if CREATE_VALIDATION_VIDEO:
        # Extract frames from the video and save them to a temporary directory
        print(f"Extracting frames for {recording_id}")
        tmp_frames_dir = tempfile.TemporaryDirectory()
        tmp_frames_path = Path(tmp_frames_dir.name)
        extract_frames_to_dir(
            video_path=trial_recording_path,
            frames_path=tmp_frames_path,
            print_output=False,
        )
        frames = sorted(list(tmp_frames_path.glob("*.jpg")), key=lambda x: int(x.stem))

        print(f"Drawing annotations for {recording_id}")
        draw_validation_video_frames(
            frames=frames,
            annotations_per_frame=annotations_per_frame,
            gaze_position_per_frame=gaze_position_per_frame,
            tracked_classes=tracked_classes,
        )

        print(f"Creating video for {recording_id}")
        cmd = f'ffmpeg -hwaccel cuda -y -pattern_type glob -framerate {TOBII_GLASSES_FPS} -i "{tmp_frames_path!s}/*.jpg" -c:v libx264 -pix_fmt yuv420p "{LABELING_VALIDATION_VIDEOS_PATH}/{recording_id}.mp4"'
        subprocess.run(
            cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
        )

ground_truth_df = pd.DataFrame(gt_rows)
ground_truth_df.to_csv(
    GROUND_TRUTH_PATH,
    index=False,
)

Extracting video frames for 67b71a70-da64-467a-9fb6-91bc29265fd1
Loading gaze data for 67b71a70-da64-467a-9fb6-91bc29265fd1
Getting viewed annotations for 67b71a70-da64-467a-9fb6-91bc29265fd1
Processing 1147 annotations for class naaldcontainer


100%|██████████| 1147/1147 [00:16<00:00, 68.96it/s]


Processing 868 annotations for class spuit


100%|██████████| 868/868 [00:09<00:00, 95.63it/s] 


Processing 1190 annotations for class keukenmes


...

Processing 98 annotations for class monitor


100%|██████████| 98/98 [00:01<00:00, 87.96it/s]


Building ground truth for b8f453aa-5a12-4cbb-a0ec-20eb503f8797
Extracting frames for b8f453aa-5a12-4cbb-a0ec-20eb503f8797
Drawing annotations for b8f453aa-5a12-4cbb-a0ec-20eb503f8797


Drawing annotations on frames: 100%|██████████| 1364/1364 [00:06<00:00, 213.82it/s]


Creating video for b8f453aa-5a12-4cbb-a0ec-20eb503f8797
